Carteira baseada no número de Graham sobre o valor intrínseco de uma ação, foca em pegar empresas que sejam lucrativas e baratas: empresas operando a P/L menor que 15 e com P/VPA menor que 1,5. Multiplicando 15 x 1,5 temos o número 22,5 que nos indica a fórmula de graham. Valor Intrínseco de uma ação = $\sqrt{22,5 \times \text{LPA} \times \text{VPA}}$.

Critérios:


- Ter Lucro por ação maior que zero ,isto é, a empresa não pode estar com prejuízo atualmente
- Ter Valor Patrimonial por ação positivo, ou seja, a empresa não pode ter mais passivos (obrigações a pagar) que ativos (bens ou direitos a receber)
- Ter volume médio de negociação diário de no mínimo R$250.000,00
- Ter lucro líquido medio positivo em todos os últimos 5 exercícios.


In [75]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True):
    
    # from IPython.display import display, HTML

    # if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
    #     df['prefixo'] = df['Papel'].astype(str).str[:4]
    #     df=df.drop_duplicates(subset='prefixo', keep='first')
    #     # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [76]:
# from DT_atualiza_settings import *
# from DT_StatusInvest import SI
# SI(mercado = 'Acoes' )

In [77]:
import os
# Caminho do arquivo local
file_path = os.path.expanduser('~/GHub/Codigos-em-financas/data/SI_Acoes.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    funds = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    funds = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL.")


Arquivo lido localmente.


In [78]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [79]:
fundsSI =  funds[ (funds[' LIQUIDEZ MEDIA DIARIA'] > 300000) &
                  (funds[' LPA'] > 0) & 
                  (funds[' VPA'] > 0) & 
                  (funds['CAGR LUCROS 5 ANOS'] > 0) ]
fundsSI

fundsSI['valor intrinseco'] = np.round(np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA']),2)
fundsSI['VALOR DE MERCADO (em B)'] = fundsSI[' VALOR DE MERCADO']  / 1e9
fundsSI['LIQUIDEZ MEDIA DIARIA (em M)'] = fundsSI[' LIQUIDEZ MEDIA DIARIA'] / 1e6
fundsSI['Delta (%)'] = np.round((fundsSI['valor intrinseco'] / fundsSI['PRECO'] -1)*100,2)
fundsSI["Rank"]   = fundsSI['Delta (%)'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

colunas_exibidas = ['TICKER','PRECO' , 'CAGR LUCROS 5 ANOS' ,'LIQUIDEZ MEDIA DIARIA (em M)',
                 ' VPA',' LPA' ,  'VALOR DE MERCADO (em B)',
                'valor intrinseco' , 'Delta (%)' ,'P/L', 'DIVIDA LIQUIDA / EBIT' ]

b_print(fundsSI[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
EUCA4,13.96,68.52,0.566966,27.46,4.25,1.356731,51.24,267.05,3.28,1.61
SOMA3,5.95,80.87,18.130278,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
ALLD3,8.42,20.06,0.458929,16.97,1.65,0.791977,25.10,198.10,5.09,0.81
BRAP3,16.66,9.00,1.366684,21.50,4.66,6.814301,47.48,184.99,3.57,-0.17
USIM3,5.99,2.17,2.117261,19.03,0.65,7.560726,16.68,178.46,9.28,0.82
USIM5,6.09,2.17,88.635140,19.03,0.65,7.560726,16.68,173.89,9.43,0.82
LOGG3,21.60,48.82,5.358980,42.29,3.61,1.908301,58.61,171.34,5.98,3.79
BRAP4,17.62,9.00,105.597653,21.50,4.66,6.814301,47.48,169.47,3.78,-0.17
JHSF3,4.45,25.39,11.826391,7.53,0.83,3.029805,11.86,166.52,5.35,2.55
VBBR3,22.32,23.45,164.197309,18.25,8.18,24.875370,57.96,159.68,2.73,0.73


outros criterios adicionais

In [80]:
'''
P/L > 0
'''
b_print(fundsSI[fundsSI['P/L'] > 0][colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
EUCA4,13.96,68.52,0.566966,27.46,4.25,1.356731,51.24,267.05,3.28,1.61
SOMA3,5.95,80.87,18.130278,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
ALLD3,8.42,20.06,0.458929,16.97,1.65,0.791977,25.10,198.10,5.09,0.81
BRAP3,16.66,9.00,1.366684,21.50,4.66,6.814301,47.48,184.99,3.57,-0.17
USIM3,5.99,2.17,2.117261,19.03,0.65,7.560726,16.68,178.46,9.28,0.82
USIM5,6.09,2.17,88.635140,19.03,0.65,7.560726,16.68,173.89,9.43,0.82
LOGG3,21.60,48.82,5.358980,42.29,3.61,1.908301,58.61,171.34,5.98,3.79
BRAP4,17.62,9.00,105.597653,21.50,4.66,6.814301,47.48,169.47,3.78,-0.17
JHSF3,4.45,25.39,11.826391,7.53,0.83,3.029805,11.86,166.52,5.35,2.55
VBBR3,22.32,23.45,164.197309,18.25,8.18,24.875370,57.96,159.68,2.73,0.73


In [81]:
'''
P/L > 0 e DIVIDA LIQUIDA / EBIT < 3
'''
fundsSI.fillna(0,inplace=True) ## bancos tem NaN
b_print(fundsSI[(fundsSI['P/L'] > 0) & (fundsSI['DIVIDA LIQUIDA / EBIT'] < 3) ][colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
EUCA4,13.96,68.52,0.566966,27.46,4.25,1.356731,51.24,267.05,3.28,1.61
SOMA3,5.95,80.87,18.130278,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
ALLD3,8.42,20.06,0.458929,16.97,1.65,0.791977,25.10,198.10,5.09,0.81
BRAP3,16.66,9.00,1.366684,21.50,4.66,6.814301,47.48,184.99,3.57,-0.17
USIM3,5.99,2.17,2.117261,19.03,0.65,7.560726,16.68,178.46,9.28,0.82
USIM5,6.09,2.17,88.635140,19.03,0.65,7.560726,16.68,173.89,9.43,0.82
BRAP4,17.62,9.00,105.597653,21.50,4.66,6.814301,47.48,169.47,3.78,-0.17
JHSF3,4.45,25.39,11.826391,7.53,0.83,3.029805,11.86,166.52,5.35,2.55
VBBR3,22.32,23.45,164.197309,18.25,8.18,24.875370,57.96,159.68,2.73,0.73
SCAR3,20.67,73.93,0.348203,29.16,4.32,1.193430,53.24,157.57,4.79,2.87


remove empresas repetidas, mantem primeira ocorrencia

In [82]:
fundsSI['prefixo'] = fundsSI['TICKER'].str[:4]
fundsSI.drop_duplicates(subset='prefixo', keep='first').head(20)
df_limpo = fundsSI.drop_duplicates(subset='prefixo', keep='first')
df_limpo = df_limpo.drop('prefixo', axis=1) 
b_print(df_limpo[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
EUCA4,13.96,68.52,0.566966,27.46,4.25,1.356731,51.24,267.05,3.28,1.61
SOMA3,5.95,80.87,18.130278,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
ALLD3,8.42,20.06,0.458929,16.97,1.65,0.791977,25.10,198.10,5.09,0.81
BRAP3,16.66,9.00,1.366684,21.50,4.66,6.814301,47.48,184.99,3.57,-0.17
USIM3,5.99,2.17,2.117261,19.03,0.65,7.560726,16.68,178.46,9.28,0.82
LOGG3,21.60,48.82,5.358980,42.29,3.61,1.908301,58.61,171.34,5.98,3.79
JHSF3,4.45,25.39,11.826391,7.53,0.83,3.029805,11.86,166.52,5.35,2.55
VBBR3,22.32,23.45,164.197309,18.25,8.18,24.875370,57.96,159.68,2.73,0.73
SCAR3,20.67,73.93,0.348203,29.16,4.32,1.193430,53.24,157.57,4.79,2.87
BMGB4,3.99,10.97,2.855856,7.44,0.63,2.327097,10.27,157.39,6.30,0.00


### outra estória de porque 22,5

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce. Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos. O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores. Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

In [83]:
import os
# Caminho do arquivo local /home/yair/GHub/Codigos-em-financas/data/SI_Stocks.csv
file_path = os.path.expanduser('/home/yair/GHub/Codigos-em-financas/data/SI_Stocks.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Stocks.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    funds = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    funds = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL.")


Arquivo lido localmente.


In [84]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [88]:
funds

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
0,A,134.35,0.70,27.32,6.54,3.51,54.39,25.03,21.75,23.74,24.45,0.71,0.20,5.94,20.68,-5.73,1.78,23.94,12.85,16.97,0.54,0.46,0.59,6.82,31.45,NaN,20.54,4.92,1.11,3.860248e+10
1,AA,40.59,0.99,-35.91,2.00,0.72,11.35,0.12,-2.65,806.66,1028.51,221.85,0.55,0.95,7.45,-1.08,1.41,-5.56,-2.01,-2.81,0.36,0.64,0.76,-4.67,NaN,NaN,20.32,-1.13,0.53,1.048662e+10
2,AACG,1.03,NaN,-3.18,1.44,0.46,51.68,-31.49,-32.34,-3.26,-2.65,0.61,-0.27,1.03,-0.98,-0.53,0.23,-45.26,-14.39,-39.89,0.32,0.68,0.44,176.03,NaN,NaN,0.72,-0.32,-0.08,3.290850e+07
3,AACQ,9.99,NaN,-93.43,180.94,1.25,NaN,NaN,NaN,-27.75,-27.74,0.01,-0.07,NaN,2094.26,-1.25,3.56,-193.66,-1.33,-652.08,0.01,0.12,0.00,NaN,NaN,NaN,0.06,-0.11,NaN,9.047194e+08
4,AACQU,11.05,NaN,-103.35,200.14,1.38,NaN,NaN,NaN,-30.69,-27.74,0.01,-0.07,NaN,2316.47,-1.38,3.56,-193.66,-1.33,-652.08,0.01,0.12,0.00,NaN,NaN,NaN,0.06,-0.11,NaN,9.047194e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5232,ZUO,9.90,NaN,-31.73,7.81,1.77,67.65,-4.91,-10.61,-68.56,-60.41,8.15,-0.93,3.36,3.46,-8.79,2.77,-24.61,-5.57,-4.07,0.23,0.77,0.52,12.93,NaN,NaN,1.27,-0.31,0.42,1.499622e+09
5233,ZVO,0.09,NaN,-0.08,0.47,0.04,35.09,-17.75,-17.80,-0.08,0.59,0.66,-4.11,0.01,-1.38,-0.07,0.95,-619.73,-47.91,-619.73,0.08,0.92,2.69,-12.86,NaN,NaN,0.19,-1.20,0.00,3.078549e+06
5234,ZYME,17.21,NaN,-10.43,3.23,2.43,100.00,-189.75,-182.75,-10.04,-7.53,2.52,-0.81,19.06,4.56,-8.27,4.10,-30.97,-23.33,-33.35,0.75,0.25,0.13,7.47,NaN,NaN,5.33,-1.65,0.07,1.185382e+09
5235,ZYNE,1.30,NaN,-1.87,2.42,1.73,NaN,NaN,NaN,-1.87,-0.91,0.96,-1.24,NaN,2.52,-59.32,3.43,-129.50,-92.85,-129.50,0.72,0.28,0.00,NaN,NaN,NaN,0.54,-0.70,4.43,7.012122e+07


In [86]:
fundsSI =  funds[ #(funds[' LIQUIDEZ MEDIA DIARIA'] > 300000) &
                  (funds[' LPA'] > 0) & 
                  (funds[' VPA'] > 0) & 
                  (fundsSI['P/L'] > 0) & 
                  (fundsSI['DIVIDA LIQUIDA / EBIT'] < 3) &
                  (funds['CAGR LUCROS 5 ANOS'] > 0) 
                  ]
fundsSI

fundsSI['valor intrinseco'] = np.round(np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA']),2)
fundsSI['VALOR DE MERCADO (em B)'] = fundsSI[' VALOR DE MERCADO']  / 1e9
fundsSI['LIQUIDEZ MEDIA DIARIA (em M)'] = fundsSI[' LIQUIDEZ MEDIA DIARIA'] / 1e6
fundsSI['Delta (%)'] = np.round((fundsSI['valor intrinseco'] / fundsSI['PRECO'] -1)*100,2)
fundsSI["Rank"]   = fundsSI['Delta (%)'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

colunas_exibidas = ['TICKER','PRECO' , 'CAGR LUCROS 5 ANOS' ,#'LIQUIDEZ MEDIA DIARIA (em M)',
                 ' VPA',' LPA' ,  'VALOR DE MERCADO (em B)',
                'valor intrinseco' , 'Delta (%)' ,'P/L', 'DIVIDA LIQUIDA / EBIT' ]

b_print(fundsSI[colunas_exibidas],n=150)

TICKER,PRECO,CAGR LUCROS 5 ANOS,VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
ACGLO,23.08,44.16,59.21,15.07,38.063916,141.69,513.91,1.53,0.25
AFGE,19.77,9.96,56.11,10.67,12.064527,116.06,487.05,1.85,1.46
ACGLP,24.99,44.16,59.21,15.07,38.063916,141.69,466.99,1.66,0.25
AFGC,22.19,9.96,56.11,10.67,12.064527,116.06,423.03,2.08,1.46
AFGB,24.43,9.96,56.11,10.67,12.064527,116.06,375.07,2.29,1.46
AAWW,102.48,9.75,107.04,12.43,2.934330,173.02,68.83,8.25,2.94
AEPPZ,47.26,2.79,50.02,4.96,49.662760,74.71,58.08,9.53,10.07
ADM,51.87,13.99,46.32,5.44,24.801226,75.30,45.17,9.54,0.72
AEPPL,52.61,2.79,50.02,4.96,49.662760,74.71,42.01,10.61,10.07
ABG,264.89,29.10,171.66,18.23,5.188533,265.35,0.17,14.53,6.58
